# Лабораторная работа 2 
Выполнил: Волков Вадим, Б01-007.
Исходный код [здесь](https://github.com/Volkov-Vad1m/ComputationalMath) 
## Ход работы
### Определение методов для решения задачи
Для начала определим класс __SlaeSolver__. Для класса определены статические методы, позволяющие решить СЛАУ разными способами. 

In [1]:
EPSILON = 1E-15
import numpy as np

class SlaeSolver :

    #Прямой ход
    def _forward_elimination(A, F): 
        n = len(A)

        for i in range(0, n):
            #выбираем элемент
            for j in range(i + 1, n):
                if A[j][i] > A[i][i]:
                    A[i], A[j] = A[j], A[i]
                    F[i], F[j] = F[j], F[i]
            
            for j in range(i + 1, n):
                F[j] -= F[i] * (A[j][i] / A[i][i])
                A[j] -= A[i] * (A[j][i] / A[i][i])

            F[i] /= A[i][i]
            A[i] /= A[i][i]

        return A, F    

    #обратный ход
    def _back_substitution(A, F):
        n = len(A)
        
        solution = [0 for _ in range(n)]
        
        # хоть и после _forward_elimination диагональные элементы нормированы,
        # все равно делаем это заново, вдруг метод будет вызван не из gauss_solve.
        solution[n-1] = F[n-1] / A[n-1][n-1]

        for i in range(n-2, -1, -1):
            solution[i] = 1 / A[i][i] * (F[i] - np.matmul(A[i], solution))

        return solution


    @staticmethod
    def gauss_solve(A, F):
        copyA = A.copy()
        copyF = F.copy()
        
        copyA, copyF = SlaeSolver._forward_elimination(copyA, copyF)

        return SlaeSolver._back_substitution(copyA, copyF)


    def _LU_decompose(A):
        n = len(A)
        L = np.zeros((n, n))
        U = np.zeros((n, n))

        for i in range(0, n):
            L[i][i] = 1

        for i in range(0, n):
            for j in range(0, n):
                sum = 0
                if i <= j:
                    for k in range(0, i):
                        sum += L[i][k] * U[k][j]
                    U[i][j] = A[i][j] - sum
                if i > j:
                    for k in range(0, j):
                        sum += L[i][k] * U[k][j]
                    L[i][j] = (A[i][j] - sum) / U[j][j]

        return L, U

    @staticmethod
    def LU_solve(A, F):
        copyF = F.copy()

        L, U = SlaeSolver._LU_decompose(A)

        SlaeSolver._forward_elimination(L, copyF)
        return SlaeSolver._back_substitution(U, copyF)


    @staticmethod
    def upper_relaxation_solve(A, F, param):
        n = len(A)
        copyA = A.copy()
        copyF = F.copy()

        L = np.zeros((n, n))
        D = np.zeros((n, n))
        U = np.zeros((n, n))

        for i in range(0,n):
            for j in range(0,n):
                if i == j:
                    D[i][j] = A[i][j]
                elif i > j:
                    L[i][j] = A[i][j]
                else:
                    U[i][j] = A[i][j]

        inv = np.linalg.inv(D + param * L)
        B = - np.matmul(inv, (param - 1) * D + param * U)
        f = np.matmul(param * inv, F)

        x = np.array([0.5 for i in range (0, n)])

    
        while np.linalg.norm(copyF - np.matmul(copyA, x)) > EPSILON:
            x = np.matmul(B, x) + f


        return x

Определим функцию __checkLU()__(уже вне класса), позволяющая узнать возможность LU разложения.

In [2]:
def checkLU(matrix):
    for i in range(0, n):
        mat_i = np.array([lines[0: i + 1] for lines in matrix[0: i + 1]])
        if not np.linalg.det(mat_i):
            return False
    return True

Определим функции, для нахождения максимального и минимального собственных чисел матрицы (Степенной метод)

In [3]:
# получить максимальное и минимальное собственные значения
def get_eigen_values(matrix):
    return get_eigen_max(matrix), 1/get_eigen_max(np.linalg.inv(matrix))


# получить максимальное собственное значение
def get_eigen_max(matrix):
    y_prev = np.array([0.1 for i in range (0, len(matrix))])
    y_cur = np.matmul(matrix, y_prev)

    eps = 1e-6
    # lim (y_cur/y_prev) = lambda_max
    while (np.linalg.norm(np.matmul(matrix, y_cur))) / np.linalg.norm( y_cur) - np.linalg.norm( y_cur) / np.linalg.norm(y_prev) > eps:
        y_prev = y_cur
        y_cur = np.matmul(matrix, y_cur)

    return np.linalg.norm(np.matmul(matrix, y_cur)) / np.linalg.norm(y_cur)

# число обусловленностей
def get_condition_number(matrix):
    return np.linalg.norm(matrix) * np.linalg.norm(np.transpose(matrix))


### Задание матрицы
Матрицы из пункта __(к)__

In [4]:
n = 10

matrix = np.zeros((n,n))

for i in range(0, n):
    for j in range(0, n):
        if i==j:
            matrix[i][j] = 1
        else: 
            matrix[i][j] = 1 / (i + j + 2)

vector = np.array( [(1 / (1+i) ) for i in range(0, n)])

print(matrix)

[[1.         0.33333333 0.25       0.2        0.16666667 0.14285714
  0.125      0.11111111 0.1        0.09090909]
 [0.33333333 1.         0.2        0.16666667 0.14285714 0.125
  0.11111111 0.1        0.09090909 0.08333333]
 [0.25       0.2        1.         0.14285714 0.125      0.11111111
  0.1        0.09090909 0.08333333 0.07692308]
 [0.2        0.16666667 0.14285714 1.         0.11111111 0.1
  0.09090909 0.08333333 0.07692308 0.07142857]
 [0.16666667 0.14285714 0.125      0.11111111 1.         0.09090909
  0.08333333 0.07692308 0.07142857 0.06666667]
 [0.14285714 0.125      0.11111111 0.1        0.09090909 1.
  0.07692308 0.07142857 0.06666667 0.0625    ]
 [0.125      0.11111111 0.1        0.09090909 0.08333333 0.07692308
  1.         0.06666667 0.0625     0.05882353]
 [0.11111111 0.1        0.09090909 0.08333333 0.07692308 0.07142857
  0.06666667 1.         0.05882353 0.05555556]
 [0.1        0.09090909 0.08333333 0.07692308 0.07142857 0.06666667
  0.0625     0.05882353 1.      

### Результаты

#### Собственные значения и число обусловленности

In [5]:
max, min = get_eigen_values(matrix)
print("\nСобственные значения степенным методом\n")
print("lambda_max:", max)
print("lambda_min:", min)


Собственные значения степенным методом

lambda_max: 2.0483598248745563
lambda_min: 0.6579603452710897


In [6]:
print("\nСобственные значения через numpy\n")
eigs, v = np.linalg.eig(matrix)
eigs.sort()
print("lambda_max:", eigs[9])
print("lambda_min:", eigs[0])


Собственные значения через numpy

lambda_max: 2.048359926977445
lambda_min: 0.6579597538101791


In [7]:
print("\nЧисло обусловленности матрицы\n")
print(get_condition_number(matrix))


Число обусловленности матрицы

11.29729295451079


In [8]:
print("\nКритерий останова: |Ax - f| < eps =", EPSILON)


Критерий останова: |Ax - f| < eps = 1e-15


#### LU метод

In [9]:
print("\nПроверка на LU\n")
print(checkLU(matrix))


Проверка на LU

True


In [10]:
print("\nLU разложение\n")
x = SlaeSolver.LU_solve(matrix, vector)
print("Корень системы:\n", x)
print("Невязка метода: ", np.linalg.norm(vector - np.matmul(matrix, x)))


LU разложение

Корень системы:
 [0.9190771092669204, 0.17554017049308804, 0.06393482401444074, 0.027274763960845518, 0.011423468535554548, 0.0035108392787171114, -0.0007899578138555328, -0.003250801449485299, -0.004697877810511589, -0.005553739941265921]
Невязка метода:  5.1925927263190304e-17


#### Метод Гаусса

In [11]:
print("Метод Гаусса\n")
x = SlaeSolver.gauss_solve(matrix, vector)
print("Корень системы:\n", x)
print("Невязка метода: ", np.linalg.norm(vector - np.matmul(matrix, x)))

Метод Гаусса

Корень системы:
 [0.9190771092669204, 0.17554017049308804, 0.06393482401444074, 0.027274763960845518, 0.011423468535554548, 0.003510839278717111, -0.0007899578138555333, -0.0032508014494852986, -0.004697877810511589, -0.005553739941265921]
Невязка метода:  5.1925927263190304e-17


#### Метод верхней релаксации

In [12]:
print("\nМетод верхней релаксации\n")
x = SlaeSolver.upper_relaxation_solve(matrix, vector, 1.5)
print("Корень системы:\n", x)
print("Невязка метода: ", np.linalg.norm(vector - np.matmul(matrix, x)))


Метод верхней релаксации

Корень системы:
 [ 9.19077109e-01  1.75540170e-01  6.39348240e-02  2.72747640e-02
  1.14234685e-02  3.51083928e-03 -7.89957814e-04 -3.25080145e-03
 -4.69787781e-03 -5.55373994e-03]
Невязка метода:  7.958884084524424e-16


#### Метод из numpy

In [13]:
print("\nМетод из numpy\n")
x = np.linalg.solve(matrix, vector)
print("Корень системы:\n", x)
print("Невязка метода: ", np.linalg.norm(vector - np.matmul(matrix, x)))


Метод из numpy

Корень системы:
 [ 9.19077109e-01  1.75540170e-01  6.39348240e-02  2.72747640e-02
  1.14234685e-02  3.51083928e-03 -7.89957814e-04 -3.25080145e-03
 -4.69787781e-03 -5.55373994e-03]
Невязка метода:  2.342840229615247e-16


Как мы видим, решения, полученные написанными мной методами, совпадают с решением через  numpy